<a href="https://colab.research.google.com/github/zakariaqasbaji/flowers_classifier/blob/main/flowers_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install kaggle
!mkdir ~/.kaggle


In [ ]:
dataset_url='https://www.kaggle.com/alxmamaev/flowers-recognition'
import opendatasets as od
od.download(dataset_url)

In [ ]:
data_dir="./flowers-recognition/flowers"
from torchvision.datasets import ImageFolder
import shutil
shutil.rmtree(data_dir+'/flowers')
import os
os.listdir(data_dir)
for clc in os.listdir(data_dir):
  print(clc,':',len(os.listdir(data_dir+"/"+clc)))


In [ ]:
dataset=ImageFolder(data_dir)

In [ ]:
len(dataset)

In [ ]:
dataset[0]

In [ ]:
dataset.classes

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
img,label=dataset[0]
plt.imshow(img)

In [ ]:
import torchvision.transforms as tt
dataset=ImageFolder(data_dir,tt.Compose([tt.Resize(64),tt.RandomCrop(64),tt.ToTensor()]))

In [ ]:
img,label=dataset[0]
plt.imshow(img.permute(1,2,0))

In [ ]:
val_pct=.1
val_size=int(val_pct*len(dataset))
train_size=len(dataset)-val_size
train_size,val_size

In [ ]:
from torch.utils.data import random_split
train_set,val_set=random_split(dataset,[train_size,val_size])
len(train_set)

In [ ]:
from torch.utils.data.dataloader import DataLoader
batch_size=128
train_dl = DataLoader(train_set, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_set, batch_size, num_workers=4, pin_memory=True)

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break
show_batch(train_dl)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device=get_default_device()
device

In [ ]:
img,label=dataset[0]
img

In [ ]:
img.shape

In [ ]:
img_gpu=to_device(img,device)
img_gpu.device

In [ ]:
train_dl=DeviceDataLoader(train_dl,device)
val_dl=DeviceDataLoader(val_dl,device)



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [ ]:
model = to_device(ResNet9(3, len(dataset.classes)), device)
model

In [ ]:
model.conv1[0].weight.device

In [ ]:
for batch in train_dl:
  images,labels=batch
  print('images.shapes: ',images.shape)
  print('images.device',images.device)
  preds=model(images)
  print('preds.shapes',preds.shape)

  break

In [ ]:
history = [evaluate(model, val_dl)]
history

In [ ]:
history+=fit(5,0.001,model,train_dl,val_dl,torch.optim.Adam)

In [ ]:
history+=fit(5,0.001,model,train_dl,val_dl,torch.optim.Adam)

In [ ]:
history+=fit(5,0.0001,model,train_dl,val_dl,torch.optim.Adam)

In [ ]:
history+=fit(5,0.0001,model,train_dl,val_dl,torch.optim.SGD)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')

plot_losses(history)

In [ ]:
history[-1]

In [ ]:
#test with individual images
def predict_image(img, model, classes):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return classes[preds[0].item()]


def show_img_prediction(img,label):
  plt.imshow(img.permute(1,2,0))
  pred=predict_image(img,model,dataset.classes)
  print('Target:',dataset.classes[label])
  print('Prediction:',pred)

show_img_prediction(*val_set[431])

In [ ]:
torch.save(model.state_dict(), 'flowers-classification')